In [51]:
import csv
import numpy as np
import pandas as pd
import json

In [123]:
KDs = pd.read_csv("Data/CFSKDsForVisualization.csv")
Modules = pd.read_csv("Data/CFSModulesForVisualization.csv")
Network = pd.read_csv("Data/CFSNetworkForVisualization.csv")
iCave_fd = pd.read_csv("Data/iCave Coordinates - Selim/cfs-net-2-fd-raw.txt", sep='\t', header=None, index_col=0)
iCave_wfd = pd.read_csv("Data/iCave Coordinates - Selim/cfs-net-2-wfd-raw.txt", sep='\t', header=None, index_col=0)

In [124]:
Network = Network.drop([17485, 17486, 17487, 17488, 17489, 17490, 17491])

In [251]:
min(list(iCave_wfd[3])), max(list(iCave_wfd[3]))

(-100, 99)

### Modules of interest

1) NKM4

2) BcellM7

3) MonoM6

4) NKM13

5) NKM9

In [125]:
NKM4 = []
BcellM7 = []
MonoM6 = []
NKM13 = []
NKM9 = []

for i in range(len(Modules)):
    if Modules["Module"][i] == "NKM4":
        NKM4.append(Modules["GeneName"][i])
    if Modules["Module"][i] == "BcellM7":
        BcellM7.append(Modules["GeneName"][i])
    if Modules["Module"][i] == "MonoM6":
        MonoM6.append(Modules["GeneName"][i])
    if Modules["Module"][i] == "NKM13":
        NKM13.append(Modules["GeneName"][i])
    if Modules["Module"][i] == "NKM9":
        NKM9.append(Modules["GeneName"][i])  

In [126]:
print(len(NKM4), len(BcellM7), len(MonoM6), len(NKM13), len(NKM9))

92 777 611 202 304


In [127]:
print(NKM4)

['ALG2', 'CLP1', 'DBNDD2', 'RGCC', 'PTGER4', 'PABPC4', 'PER1', 'EIF1', 'HSPA5', 'TAF13', 'FAM46C', 'TTC9C', 'RIPK1', 'FOS', 'ZNF674-AS1', 'TNFAIP3', 'PMAIP1', 'MIR22HG', 'RGS2', 'LARP4B', 'MTERFD2', 'SERTAD3', 'WDR53', 'TOE1', 'AC058791.2', 'RNMTL1', 'TSPYL2', 'ARL4A', 'CD83', 'BHLHE40', 'FAM177A1', 'SPRY1', 'IRF1', 'MOAP1', 'EIF1AD', 'EGR2', 'SYS1', 'BNIP1', 'YRDC', 'SRSF7', 'ZNF259', 'KPNA2', 'KANSL2', 'DHRS1', 'CSRNP1', 'ZFP36', 'B3GNT7', 'TNF', 'IRS2', 'TRAF4', 'OTUD1', 'LRRC59', 'AP006216.11', 'ZNF394', 'AVPI1', 'GRASP', 'SPATA2', 'CXCR4', 'CYTIP', 'AC005229.1', 'JUN', 'PIM3', 'DUSP10', 'PPP1R15A', 'C20orf111', 'PDE4B', 'NR4A2', 'DUSP5', 'SLAMF8', 'ATF3', 'DUSP1', 'SLC5A6', 'RP4-800J21.3', 'ZC3H12A', 'BTG2', 'TSPYL1', 'SIK1', 'TP53INP2', 'SPAG4', 'SBDS', 'MAP3K8', 'GRPEL1', 'JMJD6', 'DDIT3', 'MYLIP', 'SERTAD1', 'GADD45A', 'ISG20L2', 'MAFF', 'CTB-31O20.2', 'YPEL5', 'DNAJB1']


## Nodes creation

In [62]:
parents = np.unique(list(Network["ParentGeneName"]))
children = np.unique(list(Network["ChildGeneName"]))

nodes = list(parents) + list(children)
nodes = np.unique(nodes)

## Changing all "." to "-" to match with Selim's files for iCave ##
nodes_mod = []
for i in nodes:
    nodes_mod.append(i.replace(".", "-"))
## Changing all "." to "-" to match with Selim's files for iCave ##

In [78]:
len(nodes), len(iCave_wfd)

(13305, 13307)

In [63]:
nodes_and_groups = {}

for i in nodes_mod:
    if i in NKM4:
        nodes_and_groups[i] = "NKM4"
    elif i in BcellM7:
        nodes_and_groups[i] = "BcellM7"
    elif i in MonoM6:
        nodes_and_groups[i] = "MonoM6"
    elif i in NKM13:
        nodes_and_groups[i] = "NKM13"
    elif i in NKM9:
        nodes_and_groups[i] = "NKM9"
    else:
        nodes_and_groups[i] = "none"

## Links creation

In [64]:
links = []
for parent, child in zip(Network["ParentGeneName"], Network["ChildGeneName"]):
    ## Changing all "." to "-" to match with Selim's files for iCave ##
    parent = parent.replace(".", "-")
    child = child.replace(".", "-")
    ## Changing all "." to "-" to match with Selim's files for iCave ##
    links.append([parent, child])

In [230]:
iCave_wfd.rename(index={'100507229':'AL136115-1'}, inplace=True)

In [231]:
def Diff(li1, li2):
    return (list(list(set(li1)-set(li2)) + list(set(li2)-set(li1))))


iCave_wfd_test = list(iCave_wfd.index)
(Diff(nodes_mod, iCave_wfd_test))

['MARC1', 'PPCS']

In [232]:
without_icave = []
without_node = iCave_wfd_test

for i in nodes_mod:
    if i not in without_node:
        without_icave.append(i)
    if i in without_node:
        without_node.remove(i)

In [233]:
len(without_icave) , len(without_node)

(0, 2)

In [234]:
without_icave

[]

In [229]:
without_node

['100507229', 'MARC1', 'PPCS']

In [238]:
final = {"nodes": [], "links": []}

for node in nodes_and_groups:
    final["nodes"] += [{"id": node, "group": nodes_and_groups[node], "location": list(iCave_wfd.loc[node])}]

for parent, child in links:
    final["links"] += [{"source": parent, "target": child, "value": 1}]

In [239]:
final

{'nodes': [{'id': '61E3-4', 'group': 'none', 'location': [23, 1, -31]},
  {'id': 'A1BG-AS1', 'group': 'none', 'location': [7, 3, -92]},
  {'id': 'A2M', 'group': 'none', 'location': [27, -15, 90]},
  {'id': 'A2MP1', 'group': 'none', 'location': [26, 7, 71]},
  {'id': 'AAAS', 'group': 'none', 'location': [14, 43, -38]},
  {'id': 'AACS', 'group': 'none', 'location': [36, 52, 12]},
  {'id': 'AAED1', 'group': 'none', 'location': [-57, 38, 22]},
  {'id': 'AAGAB', 'group': 'none', 'location': [-45, -11, 24]},
  {'id': 'AAK1', 'group': 'none', 'location': [19, 32, -32]},
  {'id': 'AAMDC', 'group': 'none', 'location': [-53, 41, -22]},
  {'id': 'AAMP', 'group': 'none', 'location': [22, 13, -32]},
  {'id': 'AANAT', 'group': 'none', 'location': [58, -26, -56]},
  {'id': 'AAR2', 'group': 'none', 'location': [22, 34, -22]},
  {'id': 'AARS', 'group': 'NKM9', 'location': [-20, -16, -42]},
  {'id': 'AARS2', 'group': 'none', 'location': [-5, 34, -16]},
  {'id': 'AARSD1', 'group': 'none', 'location': [49

In [240]:
with open('complete_data_icave_format.json', 'w') as fp:
    json.dump(final, fp)

________

# Modules

### BcellM7

In [247]:
BcellM7_KD = {}
for kd, gene in zip(KDs["BcellM7_KD"], KDs["geneName_KD"]):
    BcellM7_KD[gene] = kd

len(BcellM7_KD), len(nodes)

(904, 13305)

In [248]:
## THERE IS NOT INFORMATION ABOUT KD FOR ALL NODES. I SET THEM TO 0 IF NON EXISTENT IN THE KD DOCUMENT

for n in nodes:
    if n not in BcellM7_KD:
        BcellM7_KD[n] = 0

len(BcellM7_KD), len(nodes)

(13307, 13305)

In [249]:
links_BcellM7 = []
nodes_BcellM7 = []

for parent, child in links:
    if nodes_and_groups[parent] == "BcellM7" or nodes_and_groups[child] == "BcellM7":
        links_BcellM7.append([parent, child])
        nodes_BcellM7.append([parent, nodes_and_groups[parent], BcellM7_KD[parent]])
        nodes_BcellM7.append([child, nodes_and_groups[child], BcellM7_KD[child]]) 
        
nodes_BcellM7 = set(map(tuple, nodes_BcellM7))

In [250]:
BcellM7_final = {"nodes": [], "links": []}

for node, group, kd in nodes_BcellM7:
    BcellM7_final["nodes"] += [{"id": node, "group": group, "kd": kd}]

for parent, child in links_BcellM7:
    BcellM7_final["links"] += [{"source": parent, "target": child, "value": 1}]

In [251]:
with open('BcellM7_final.json', 'w') as fp:
    json.dump(BcellM7_final, fp)

### MonoM6


In [252]:
MonoM6_KD = {}
for kd, gene in zip(KDs["MonoM6_KD"], KDs["geneName_KD"]):
    MonoM6_KD[gene] = kd

len(MonoM6_KD), len(nodes)

(904, 13305)

In [253]:
## THERE IS NOT INFORMATION ABOUT KD FOR ALL NODES. I SET THEM TO 0 IF NON EXISTENT IN THE KD DOCUMENT

for n in nodes:
    if n not in MonoM6_KD:
        MonoM6_KD[n] = 0

len(MonoM6_KD), len(nodes)

(13307, 13305)

In [254]:
links_MonoM6 = []
nodes_MonoM6 = []

for parent, child in links:
    if nodes_and_groups[parent] == "MonoM6" or nodes_and_groups[child] == "MonoM6":
        links_MonoM6.append([parent, child])
        nodes_MonoM6.append([parent, nodes_and_groups[parent], MonoM6_KD[parent]])
        nodes_MonoM6.append([child, nodes_and_groups[child], MonoM6_KD[child]])    

nodes_MonoM6 = set(map(tuple, nodes_MonoM6))

In [255]:
MonoM6_final = {"nodes": [], "links": []}

for node, group, kd in nodes_MonoM6:
    MonoM6_final["nodes"] += [{"id": node, "group": group, "kd": kd}]

for parent, child in links_MonoM6:
    MonoM6_final["links"] += [{"source": parent, "target": child, "value": 1}]

In [256]:
with open('MonoM6_final.json', 'w') as fp:
    json.dump(MonoM6_final, fp)

### NKM13

In [257]:
NKM13_KD = {}
for kd, gene in zip(KDs["NKM13_KD"], KDs["geneName_KD"]):
    NKM13_KD[gene] = kd

len(NKM13_KD), len(nodes)

(904, 13305)

In [258]:
## THERE IS NOT INFORMATION ABOUT KD FOR ALL NODES. I SET THEM TO 0 IF NON EXISTENT IN THE KD DOCUMENT

for n in nodes:
    if n not in NKM13_KD:
        NKM13_KD[n] = 0

len(NKM13_KD), len(nodes)

(13307, 13305)

In [259]:
links_NKM13 = []
nodes_NKM13 = []

for parent, child in links:
    if nodes_and_groups[parent] == "NKM13" or nodes_and_groups[child] == "NKM13":
        links_NKM13.append([parent, child])
        nodes_NKM13.append([parent, nodes_and_groups[parent], NKM13_KD[parent]])
        nodes_NKM13.append([child, nodes_and_groups[child], NKM13_KD[child]]) 

nodes_NKM13 = set(map(tuple, nodes_NKM13))

In [260]:
NKM13_final = {"nodes": [], "links": []}

for node, group, kd in nodes_NKM13:
    NKM13_final["nodes"] += [{"id": node, "group": group, "kd": kd}]

for parent, child in links_NKM13:
    NKM13_final["links"] += [{"source": parent, "target": child, "value": 1}]

In [261]:
with open('NKM13_final.json', 'w') as fp:
    json.dump(NKM13_final, fp)

### NKM9

In [262]:
NKM9_KD = {}
for kd, gene in zip(KDs["NKM9_KD"], KDs["geneName_KD"]):
    NKM9_KD[gene] = kd

len(NKM9_KD), len(nodes)

(904, 13305)

In [263]:
## THERE IS NOT INFORMATION ABOUT KD FOR ALL NODES. I SET THEM TO 0 IF NON EXISTENT IN THE KD DOCUMENT

for n in nodes:
    if n not in NKM9_KD:
        NKM9_KD[n] = 0

len(NKM9_KD), len(nodes)

(13307, 13305)

In [264]:
links_NKM9 = []
nodes_NKM9 = []

for parent, child in links:
    if nodes_and_groups[parent] == "NKM9" or nodes_and_groups[child] == "NKM9":
        links_NKM9.append([parent, child])
        nodes_NKM9.append([parent, nodes_and_groups[parent], NKM9_KD[parent]])
        nodes_NKM9.append([child, nodes_and_groups[child], NKM9_KD[child]])  

nodes_NKM9 = set(map(tuple, nodes_NKM9))

In [265]:
NKM9_final = {"nodes": [], "links": []}

for node, group, kd in nodes_NKM9:
    NKM9_final["nodes"] += [{"id": node, "group": group, "kd": kd}]

for parent, child in links_NKM9:
    NKM9_final["links"] += [{"source": parent, "target": child, "value": 1}]

In [266]:
with open('NKM9_final.json', 'w') as fp:
    json.dump(NKM9_final, fp)

### NKM4

In [267]:
NKM4_KD = {} ## THIS SPECIFIC MODULE DONT HAVE KD.
#for kd, gene in zip(KDs["BcellM7_KD"], KDs["geneName_KD"]):
#    BcellM7_KD[gene] = kd

#len(BcellM7_KD), len(nodes)

In [268]:
## THERE IS NOT INFORMATION ABOUT KD FOR ALL NODES. I SET THEM TO 0 IF NON EXISTENT IN THE KD DOCUMENT

for n in nodes:
    if n not in NKM4_KD:
        NKM4_KD[n] = 0

len(NKM4_KD), len(nodes)

(13305, 13305)

In [269]:
links_NKM4 = []
nodes_NKM4 = []

for parent, child in links:
    if nodes_and_groups[parent] == "NKM4" or nodes_and_groups[child] == "NKM4":
        links_NKM4.append([parent, child])
        nodes_NKM4.append([parent, nodes_and_groups[parent], NKM4_KD[parent]])
        nodes_NKM4.append([child, nodes_and_groups[child], NKM4_KD[child]]) 
        
nodes_NKM4 = set(map(tuple, nodes_NKM4))

In [270]:
NKM4_final = {"nodes": [], "links": []}

for node, group, kd in nodes_NKM4:
    NKM4_final["nodes"] += [{"id": node, "group": group, "kd": kd}]

for parent, child in links_NKM4:
    NKM4_final["links"] += [{"source": parent, "target": child, "value": 1}]

In [271]:
with open('NKM4_final.json', 'w') as fp:
    json.dump(NKM4_final, fp)